In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from time import time

RANSAC_ITER = 300

RANSAC_THERSHOLD = 5

In [2]:
raw_img1= cv2.imread("input/img01.jpg")
raw_img2= cv2.imread("input/img02.jpg")

sift=cv2.xfeatures2d.SIFT_create(200)
kps1 ,des1= sift.detectAndCompute(raw_img1,None)
kps2 ,des2= sift.detectAndCompute(raw_img2,None)
bf = cv2.BFMatcher()
match_list = bf.knnMatch(des1,des2,2)


In [7]:
good_match_list = [m for m, n in match_list if m.distance < 0.75 * n.distance]

src_pts = np.float32([kps1[m.queryIdx].pt for m in good_match_list])
dst_pts = np.float32([kps2[m.trainIdx].pt for m in good_match_list])


In [8]:
src_pts.shape

(136, 2)

In [38]:
def random_partition(data):
    '''
    randomly select two points     
    '''
    np.random.shuffle(data)
    twoPoints = data[:4]
    samplePoint = data[4:]
    return twoPoints,samplePoint


def H_from_points(fp,tp):
    """ DLT algo"""
    assert fp.shape == tp.shape
    h = fp.shape[0]
    A = np.zeros((2*h,9))
    for i in range(h):
        x,y,u,v=fp[i,0],fp[i,1],tp[i,0],tp[i,1]
        A[2*i]= np.array([x,y,1,0,0,0,-u*x,-u*y,-u])
        A[2*i+1]=np.array([0,0,0,x,y,1,-v*x,-v*y,-v])
    U,D,V = np.linalg.svd(A)
    H=V[8].reshape(3,3)
    return H/H[-1,-1]
    
    

def conliner4(a):
    p1,p2,p3,p4=a
    return conliner3(p1,p2,p3) or conliner3(p1,p2,p4) or conliner3(p2,p3,p4)
    

def conliner3(p1,p2,p3):
    x1,y1=p1
    x2,y2=p2
    x3,y3=p3
    return abs((y2-y1)*(x3-x1)-(y3-y1)*(x2-x1)) < 1e-12

def homo_transform(pt,H):
    _pt = np.append(pt,1)
    _rs= np.dot(H,_pt)
    _rs/=_rs[-1]
    return _rs[:2]


def myfindHomo_ransac(src_pts,dst_pts):
    comb = list(zip(src_pts,dst_pts))
    good_inlier_mask = None
    best_inlier=-1
    
    #find good_mask and inlier
    for _ in range(RANSAC_ITER):
        a,b = random_partition(comb)
        while conliner4([ pair[0] for pair in a]):
            a,b = random_partition(comb)
        mask = H_from_points(np.array([pair[0] for pair in a]),np.array([pair[1] for pair in a] ))
        
        _inlier=0
        for src,dst in b:
            trans_dst = homo_transform(src,mask)
            diff = np.sqrt(sum((dst-trans_dst)**2))
            if diff <=RANSAC_THERSHOLD:
                _inlier +=1
        if _inlier > best_inlier:
            good_inlier_mask = mask
            best_inlier = _inlier
            
    inlier_src=[]
    inlier_dst=[]
    for src,dst in comb:
            trans_dst = homo_transform(src,good_inlier_mask)
            diff = np.sqrt(np.sum(np.square(dst-trans_dst)))
            if diff <=RANSAC_THERSHOLD:
                inlier_src.append(src)
                inlier_dst.append(dst)
    
    # find the best_mask
    best_mask=[]
    best_diff=np.float("Inf")
    comb = list(zip(inlier_src,inlier_dst))
    for _ in range(RANSAC_ITER):
        a,b = random_partition(comb)
        while conliner4([ pair[0] for pair in a]):
            a,b = random_partition(comb)
        mask = H_from_points(np.array([pair[0] for pair in a]),np.array([pair[1] for pair in a] ))
        
        diff=0
        for src,dst in b:
            trans_dst = homo_transform(src,mask)
            diff += np.sqrt(sum((dst-trans_dst)**2))
        if diff < best_diff:
            best_diff = diff
            best_mask = mask
    print(best_diff)
    return best_mask
           
          
        

    
        
    
    

In [10]:
t1= time()

mask=myfindHomo_ransac(src_pts,dst_pts)
print(time()-t1)


161.632511904
1.2048048973083496


In [38]:
# def mywarpPerspective(img,mask,size):
#     new_img = np.zeros([size[1],size[0],3],dtype=np.uint8)
#     print(new_img.shape)
#     for i in range(size[1]):
#         for j in range(size[0]):
#             new_y,new_x = homo_transform([j,i],mask)
#             new_x = int(round(new_x))
#             new_y = int(round(new_y))
#             if 0<=new_x < size[1] and 0<=new_y< size[0]:
#                 new_img[new_x][new_y]=img[i][j]
#     return new_img    

In [7]:
def mywarpPerspective(img,M,size):
    new_img = np.zeros([size[1],size[0],3],dtype=np.uint8)
    print(new_img.shape)
    for i in range(size[1]):
        for j in range(size[0]):
            
            new_x = np.divide(M[1,0]*j + M[1,1]*i+M[1,2],M[2,0]*j+M[2,1]*i+M[2,2])
            new_y = np.divide(M[0,0]*j + M[0,1]*i+M[0,2],M[2,0]*j+M[2,1]*i+M[2,2])      
            
            new_x = int(round(new_x))
            new_y = int(round(new_y))
            if 0<=new_x < size[1] and 0<=new_y< size[0]:
                new_img[new_x][new_y]=img[i][j]
                
                
    
    return new_img    
   

In [588]:
t2=time()
nim = mywarpPerspective(raw_img1,mask,(size[1],size[0]))
print(time()-t2)

(5472, 3648, 3)
169.71235752105713


In [589]:
cv2.imwrite("step3.jpg",nim)

True

In [17]:
def calculate_size(size_image1, size_image2, homography):
    (h1, w1) = size_image1[:2]
    (h2, w2) = size_image2[:2]
    # remap the coordinates of the projected image onto the panorama image space
    top_left = np.dot(homography, np.asarray([0, 0, 1]))
    top_right = np.dot(homography, np.asarray([w2, 0, 1]))
    bottom_left = np.dot(homography, np.asarray([0, h2, 1]))
    bottom_right = np.dot(homography, np.asarray([w2, h2, 1]))

    # normalize
    top_left = top_left / top_left[2]
    top_right = top_right / top_right[2]
    bottom_left = bottom_left / bottom_left[2]
    bottom_right = bottom_right / bottom_right[2]


    pano_left = int(min(top_left[0], bottom_left[0], 0))
    pano_right = int(max(top_right[0], bottom_right[0], w1))
    W = pano_right - pano_left

    pano_top = int(min(top_left[1], top_right[1], 0))
    pano_bottom = int(max(bottom_left[1], bottom_right[1], h1))
    H = pano_bottom - pano_top

    size = (W, H)

    # offset of first image relative to panorama
    X = int(min(top_left[0], bottom_left[0], 0))
    Y = int(min(top_left[1], top_right[1], 0))
    offset = (-X, -Y)
    
    ## Update the homography to shift by the offset
    # does offset need to be remapped to old coord space?
    # print homography
    # homography[0:2,2] += offset
    return (size, offset)


In [18]:

def merge_images_translation(image1, image2, offset):
    
    (h1, w1) = image1.shape[:2]
    (h2, w2) = image2.shape[:2]
    (ox, oy) = offset
    ox = int(ox)
    oy = int(oy)
    oy = 0
  
    image = np.zeros((h1+oy, w1+ox, 3), np.uint8)
  
    image[:h1, :w1] = image1
    image[:h2, ox:ox+w2] = image2
  
    return image
 

In [19]:
size,offset = calculate_size(raw_img1.shape,raw_img2.shape,mask)

In [20]:
immmmmm = merge_images_translation(raw_img1,raw_img2,offset)

In [21]:
cv2.imwrite("mmmm.jpg",immmmmm)

True

In [6]:

imgs = imgs[:]
centerIdx = len(imgs)/2

left_img = cv2.imread(imgs[0])
    
for i in range(1, centerIdx+1):
    right_img = cv2.imread(imgs[i])
    H = getHomoByImage(left_img, right_img)
    
    xmin, xmax, ymin, ymax, T = getCoorTrans(left_img, H)
    
    new_H = np.matmul(T, H)
    
    offsety = abs(int(-ymin))
    offsetx = abs(int(-xmin))

    left_img = cv2.warpPerspective(left_img, new_H, (int(xmax-xmin),int(ymax-ymin)))
    
    y_diff = right_img.shape[0]+offsety - left_img.shape[0]
    
    if y_diff >= 0:
        shape = list(left_img.shape)
        shape[0] = y_diff+1
        shape = tuple(shape)
        blank_image = np.zeros(shape, np.uint8)
        left_img = np.concatenate((left_img, blank_image), axis=0)
        #left_img = np.vstack((left_img, np.zeros((left_img.shape[0], y_diff+1), dtype=left_img.dtype)))
    x_diff = right_img.shape[1]+offsetx - left_img.shape[1]
    if x_diff >= 0:
        shape = list(left_img.shape)
        shape[1] = x_diff+1
        shape = tuple(shape)
        blank_image = np.zeros(shape, np.uint8)
        left_img = np.concatenate((left_img, blank_image), axis=1)
    #right_img[offsety:left_img.shape[0]+offsety, offsetx:left_img.shape[1]+offsetx] = left_img
    for i in range(offsety,right_img.shape[0]+offsety):
        for j in range(offsetx,right_img.shape[1]+offsetx):
            if np.array_equal(right_img[i-offsety, j-offsetx], np.array([0,0,0])):
                continue
            left_img[i, j] = right_img[i-offsety, j-offsetx]


NameError: name 'imgs' is not defined

In [13]:
def calculate_size(size_image1, size_image2, homography):
  
    (h1, w1) = size_image1[:2]
    (h2, w2) = size_image2[:2]
    top_left = np.dot(homography,np.array([0,0,1]))
    top_right = np.dot(homography,np.array([w2,0,1]))
    bottom_left = np.dot(homography,np.array([0,h2,1]))
    bottom_right = np.dot(homography,np.array([w2,h2,1]))

    top_left = top_left/top_left[2]
    top_right = top_right/top_right[2]
    bottom_left = bottom_left/bottom_left[2]
    bottom_right = bottom_right/bottom_right[2]

    pano_left = int(min(top_left[0], bottom_left[0], 0))
    pano_right = int(max(top_right[0], bottom_right[0], w1))
    
    W = pano_right - pano_left

    pano_top = int(min(top_left[1], top_right[1], 0))
    pano_bottom = int(max(bottom_left[1], bottom_right[1], h1))
    H = pano_bottom - pano_top
    
    size = (W, H)

    X = int(min(top_left[0], bottom_left[0], 0))
    Y = int(min(top_left[1], top_right[1], 0))
    offset = (-X, -Y)

    return (size, offset)

In [30]:
def merge_images(image1, image2, homography, size, offset):

    (h1, w1) = image1.shape[:2]
    (h2, w2) = image2.shape[:2]

    panorama = np.zeros((size[1], size[0], 3), np.uint8)

    (ox, oy) = offset

    translation = np.matrix([[1.0, 0.0, ox],[0, 1.0, oy],[0.0, 0.0, 1.0]])
    homography = translation * homography
    cv2.warpPerspective(image2, homography, size, panorama)

    panorama[oy:h1+oy, ox:ox+w1] = image1  

    return panorama

In [83]:
size,offset = calculate_size(raw_img1.shape,raw_img2.shape,mask)

In [71]:
def getCoorTrans(img, M):
    xmin = float('Inf')
    xmax = float('-Inf')
    ymin = float('Inf')
    ymax = float('-Inf')
    

    corner0 = homo_transform([0, 0], M)
    corner0[0] = int(corner0[0])
    corner0[1] = int(corner0[1])
    a, b = corner0
    xmin = a if a < xmin else xmin
    xmax = a if a > xmax else xmax
    ymin = b if b < ymin else ymin
    ymax = b if b > ymax else ymax

    corner1 = homo_transform([img.shape[1], 0], M)
    corner1[0] = int(corner1[0])
    corner1[1] = int(corner1[1])
    a, b = corner1
    xmin = a if a < xmin else xmin
    xmax = a if a > xmax else xmax
    ymin = b if b < ymin else ymin
    ymax = b if b > ymax else ymax


    corner2 = homo_transform([0, img.shape[0]], M)
    corner2[0] = int(corner2[0])
    corner2[1] = int(corner2[1])
    a, b = corner2
    xmin = a if a < xmin else xmin
    xmax = a if a > xmax else xmax
    ymin = b if b < ymin else ymin
    ymax = b if b > ymax else ymax



    corner3 = homo_transform([img.shape[1], img.shape[0]], M)
    corner3[0] = int(corner3[0])
    corner3[1] = int(corner3[1])
    a, b = corner3
    xmin = a if a < xmin else xmin
    xmax = a if a > xmax else xmax
    ymin = b if b < ymin else ymin
    ymax = b if b > ymax else ymax

    T = np.zeros(shape=(3,3))
    T[0] = [1, 0, 0]
    T[1] = [0, 1, 0]
    T[2] = [-xmin, -ymin, 1]

    return xmin, xmax, ymin, ymax, T.transpose()


In [72]:
getCoorTrans(raw_img1,mask)


(-377.0, 3347.0, -165.0, 5669.0, array([[   1.,    0.,  377.],
        [   0.,    1.,  165.],
        [   0.,    0.,    1.]]))

In [75]:
def CoorTrans(img, M):
    min_x,min_y,max_x,max_y = float('Inf'),float('Inf'),float("-Inf"),float("-Inf")
    (h1, w1) = img.shape[:2]
    top_left = homo_transform([0, 0], M)
    top_right = homo_transform([w1, 0], M)
    bottom_left = homo_transform([0, h1], M)
    bottom_right = homo_transform([w1, h1], M)
    
    for corner in [top_left,top_right,bottom_left,bottom_right]:
        a, b = corner[0],corner[1]
        min_x = a if a < min_x else min_x
        max_x = a if a > max_x else max_x
        min_y = b if b < min_y else min_y
        max_y = b if b > max_y else max_y
    T = np.zeros(shape=(3,3))
    T[0] = [1, 0, 0]
    T[1] = [0, 1, 0]
    T[2] = [-min_x, -min_y, 1]

    return int(min_x),int(max_x), int(min_y),int(max_y),T.transpose()

In [76]:
CoorTrans(raw_img1,mask)

(-377, 3347, -165, 5669, array([[   1.        ,    0.        ,  377.16807868],
        [   0.        ,    1.        ,  165.91598705],
        [   0.        ,    0.        ,    1.        ]]))

In [78]:
def merge_images_translation(image1, image2, offset):

    ## Put images side-by-side into 'image'.
    (h1, w1) = image1.shape[:2]
    (h2, w2) = image2.shape[:2]
    (ox, oy) = offset
    ox = int(ox)
    oy = int(oy)
    oy = 0

    image = np.zeros((h1+oy, w1+ox, 3), np.uint8)

    image[:h1, :w1] = image1
    image[:h2, ox:ox+w2] = image2
  
    return image


In [80]:
immmm= merge_images_translation(raw_img1,raw_img2,offset)

In [85]:
panorama = merge_images(raw_img1, raw_img2, mask, size, offset)
